In [24]:
!pip install -q datasets soundfile speechbrain requests
!pip install -q transformers
!pip install -q --upgrade accelerate

In [1]:
from pathlib import Path
import multiprocessing
import requests

from datasets import load_dataset, Audio, Dataset, DatasetDict

In [2]:
n_cpu = multiprocessing.cpu_count(); n_cpu
data_path = Path.cwd() / "data"
audio_path = data_path / "audio"
audio_path.mkdir(parents=True, exist_ok=True)

## Load the dataset

In [3]:
dataset_name = "openpecha/tts-training-filtered"
ds = load_dataset(dataset_name)
ds

DatasetDict({
    train: Dataset({
        features: ['file_name', 'uni', 'url', 'sentence', 'label', 'speaker_id', 'duration'],
        num_rows: 342800
    })
    test: Dataset({
        features: ['file_name', 'uni', 'url', 'sentence', 'label', 'speaker_id', 'duration'],
        num_rows: 85701
    })
})

In [10]:
def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for any HTTP errors
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    except requests.exceptions.RequestException as e:
        raise e

def prepare_audio_file(item):
    file_name = f"{item['file_name']}.wav"
    local_path = audio_path / file_name
    if not local_path.is_file():
        if item["file_name"].startswith("STT_AB"):
            url = f"https://d38pmlk0v88drf.cloudfront.net/AB_wav16k_cleaned/{file_name}"
        else:
            url = item["url"]
        try:
            download_image(url, local_path)
        except:
            local_path = None

    item["path"] = str(local_path) if local_path else local_path
    del item["file_name"]
    del item["url"]

    return item

In [5]:
# sample_ds = DatasetDict()

# sample_ds["train"] = ds["train"].select(range(8)).map(prepare_audio_file, num_proc=n_cpu-1)
# sample_ds["test"] = ds["test"].select(range(2)).map(prepare_audio_file, num_proc=n_cpu)
# print(len(sample_ds["train"]), len(sample_ds["test"]))
# sample_ds = sample_ds.filter(lambda x: x["path"], num_proc=n_cpu)
# print(len(sample_ds["train"]), len(sample_ds["test"]))

# dataset_path = data_path / "tts-training-processed-sample"
# sample_ds.save_to_disk(dataset_path)

<!-- loaded_ds = DatasetDict.load_from_disk(dataset_path)
loaded_ds -->

In [6]:
# xs = ds["test"].filter(lambda x: x["file_name"].startswith("STT_AB"), num_proc=n_cpu)

In [7]:
# xs[100]

## Download All Audio files

In [9]:
all_ds = DatasetDict()

all_ds["train"] = ds["train"].map(prepare_audio_file, num_proc=n_cpu-1)
all_ds["test"] = ds["test"].map(prepare_audio_file, num_proc=n_cpu-1)
print(len(all_ds["train"]), len(all_ds["test"]))
all_ds = all_ds.filter(lambda x: x["path"], num_proc=n_cpu)
print(len(all_ds["train"]), len(all_ds["test"]))

dataset_path = data_path / "tts-training-processed"
all_ds.save_to_disk(dataset_path)

Map (num_proc=63):   0%|          | 0/342800 [00:00<?, ? examples/s]

TimeoutError: 